In [ ]:
import tensorflow as tf

import pathlib
from typing import Any,Union

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def get_dataset_dir(path) -> str:
    return pathlib.Path(path)

In [ ]:
BATCH_SIZE:int  = 64
IMG_HEIGHT:int  = 180
IMG_WIDTH:int   = 180
VAL_SPLIT:float = 0.2

In [ ]:
def load_dataset(dir) -> Union[list | Any, list | Any]:
  train_ds = tf.keras.utils.image_dataset_from_directory(
    dir,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE)
  val_ds = tf.keras.utils.image_dataset_from_directory(
    dir,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE)
  return train_ds, val_ds

# Import dataset

In [ ]:
data_dir:str = get_dataset_dir('dataset')
print("Nombres d'image: {}".format(len(list(data_dir.glob('*/*.jpg')))))

In [ ]:
train_ds, val_ds = load_dataset(data_dir)
class_names = train_ds.class_names
print(class_names)

# Configure the dataset for performance

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(10).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
import csv

model_name = 'model_09.20.2022_20:31:25'
model:tf.keras.Model = tf.keras.models.load_model(model_name)

In [ ]:
EPOCHES=2

history:tf.keras.callbacks.History = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=EPOCHES,
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Save

In [ ]:
import datetime
import pandas as pd

date:datetime = datetime.datetime.now().strftime("%m.%d.%Y_%H:%M:%S")
model_dir:str = f"model_{date}"
model.save(model_dir)
f = open(f"{model_dir}/model_summary.txt", "a")
model.summary(print_fn=lambda x: f.write(x + '\n'))
f.close()

f = open(f"{model_dir}/model_history.csv", "a")

if pathlib.Path(f"{model_name}/model_history.csv").is_file():
    f2 = open(f"{model_name}/model_history.csv", "r")
    f.write(f2.read())
    f2.close()
f.write(pd.DataFrame.from_dict(history.history).to_csv(index=False))
f.close()

tf.keras.utils.plot_model(model, to_file=f"{model_dir}/model_plot.png", show_shapes=True, show_layer_names=True)

model_name = model_dir